In [1]:
import requests
import pandas as pd
from io import BytesIO
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [2]:
jan_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet'
feb_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'

In [3]:
df_jan = pd.read_parquet(BytesIO(requests.get(jan_url, allow_redirects=True).content))
df_feb = pd.read_parquet(BytesIO(requests.get(feb_url, allow_redirects=True).content))

In [7]:
df_jan.columns.size

19

In [8]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds()/60
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds()/60
df_jan.query('duration >= 1 and duration <= 60').shape[0]/df_jan.shape[0]

0.9827547930522406

In [9]:
df_jan.duration.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

In [13]:
categorical = ['PULocationID', 'DOLocationID']
             

In [35]:
def read_data(df):
    df = df.query('duration >= 1 and duration <= 60')
    df[categorical] = df[categorical].astype('str')
    train_dict = df[categorical].to_dict(orient='records')
    Y_train = df['duration']
    return train_dict, Y_train

In [36]:
X_train, Y_train = read_data(df_jan)
X_val, Y_val = read_data(df_feb)

/tmp/ipykernel_8934/1582843255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype('str')
/tmp/ipykernel_8934/1582843255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype('str')


In [37]:
dv = DictVectorizer()
X_train = dv.fit_transform(X_train)
X_val = dv.transform(X_val)

In [38]:
X_train.shape

(2421440, 515)

In [39]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

LinearRegression()

In [40]:
mean_squared_error(Y_train, lr.predict(X_train), squared=False)

6.9861907482152406

In [41]:
mean_squared_error(Y_val, lr.predict(X_val), squared=False)

7.786406459362245